In [166]:
import pandas as pd
from numpy.random import normal, binomial, seed
import random
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

seed(42)
random.seed(42)  # set seed for reproducibility

In [167]:
columns = ["case_id", "activity", "event_order", "exchange_rate"]
event_logs = pd.DataFrame(columns=columns)

In [168]:
def generate_activity(case_id: int, name: str, event_order=1) -> pd.DataFrame:
    """
    Generates an event log for the "Check-in"-activity.
    """
    return pd.DataFrame(
        {
            "case_id": [case_id],
            "activity": [name],
            "event_order": [event_order],
            "exchange_rate": [None],
        }
    )


print("example row: \n", generate_activity(1, "Check-in"))  # check whether it works
print("\n--------------------------------------------\n")
print("example row: \n", generate_activity(1, "Pays", 3))  # check whether it works
print("\n--------------------------------------------\n")


def generate_exchange_rate(case_id: int, event_order: int) -> pd.DataFrame:
    """
    Generates an event log for the "Check exchange rate"-activity.
    Retrieved exchange rates are sampled from normal distribution w/ mean 1 std 0.1
    """
    rate = round(normal(1, 0.1), 2)
    return pd.DataFrame(
        {
            "case_id": [case_id],
            "activity": ["Check exchange rate"],
            "event_order": [event_order],
            "exchange_rate": [rate],
        }
    )


print(
    "example row for Check exchange rate: \n", generate_exchange_rate(1, 3)
)  # check whether it works
print("\n--------------------------------------------\n")

example row: 
    case_id  activity  event_order exchange_rate
0        1  Check-in            1          None

--------------------------------------------

example row: 
    case_id activity  event_order exchange_rate
0        1     Pays            3          None

--------------------------------------------

example row for Check exchange rate: 
    case_id             activity  event_order  exchange_rate
0        1  Check exchange rate            3           1.05

--------------------------------------------



In [169]:
def generate_case(case_id: int) -> pd.DataFrame:
    """
    this function combines the different generators from above to create an artifical process instance.
    """
    event_order = 1
    case = generate_activity(case_id, "Check-in", event_order)
    event_order += 1
    # randomly decide whether guest has enough cash
    if random.uniform(0, 1) > 0.9:
        payment = generate_activity(case_id, "Pays", event_order)
        case = pd.concat([case, payment])
        return case

    exchange_rate = generate_exchange_rate(case_id, 2)
    case = pd.concat([case, exchange_rate])
    event_order += 1

    while True:
        # idea:
        # sample from binomial distribution in a way, so that higher exchange rates increase the success probability
        # to achieve this, set success probabilty of binomial distribution to exchange rate - mean(exchange rate) + 0.3
        success = exchange_rate.loc[0, "exchange_rate"] - 1
        success_probability = (
            success + 0.3 if success > -0.3 else 0
        )  # prevent from success probabilities < 0 as binomial only supports 0 <= s <= 1
        does_exchange = binomial(1, success_probability)

        # if the guest does exchange, add the follow-up activities and end the process.
        # else continue
        if does_exchange:
            withdraw = generate_activity(case_id, "Withdraws", event_order)
            event_order += 1
            payment = generate_activity(case_id, "Pays", event_order)
            return pd.concat([case, withdraw, payment])

        exchange_rate = generate_exchange_rate(case_id, event_order)
        case = pd.concat([case, exchange_rate])
        event_order += 1


print("example process instance: \n", generate_case(1))

example process instance: 
    case_id             activity  event_order  exchange_rate
0        1             Check-in            1            NaN
0        1  Check exchange rate            2           0.99
0        1            Withdraws            3            NaN
0        1                 Pays            4            NaN


In [170]:
# check whether our data generation works in principle

data = pd.DataFrame()
for case_id in range(1, 100):
    data = pd.concat([data, generate_case(case_id)])

print("Number of generated rows: ", len(data))
display(data.head(5))
display(data.tail(5))

Number of generated rows:  553


,case_id,activity,event_order,exchange_rate
0,1,Check-in,1,NaN
0,1,Check exchange rate,2,0.89
0,1,Check exchange rate,3,1.03
0,1,Check exchange rate,4,1.03
0,1,Withdraws,5,NaN


,case_id,activity,event_order,exchange_rate
0,99,Check exchange rate,5,0.94
0,99,Check exchange rate,6,0.78
0,99,Check exchange rate,7,1.09
0,99,Withdraws,8,NaN
0,99,Pays,9,NaN


In [171]:
# generate the whole dataset (takes a bit longer)

data = pd.DataFrame()

for case_id in range(1, 10000):
    data = pd.concat([data, generate_case(case_id)])

data.to_csv("artificial_data.csv", index=False)

In [175]:
print(
    "Number of instances where guest payed immediately: ",
    len(data[(data["activity"] == "Pays") & (data["event_order"] == 2)]),
)
print(
    "Number of instances where guest had to exchange money: ",
    len(data[(data["activity"] == "Check exchange rate") & (data["event_order"] == 2)]),
)

Number of instances where guest payed immediately:  984
Number of instances where guest had to exchange money:  9015
